In [20]:
using JuMP
import Pkg
Pkg.add("Gurobi")
import Gurobi
using CSV, DataFrames
using Random
import Test


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [21]:
data = CSV.read("RoofDummyData.csv",DataFrame)
data
#nrow(data) = 24
#ncol(data) = 5
data1= CSV.read("/Users/axgelx/MSO ESA prj/mso/dummydata1.csv",DataFrame)
data1
#nrow(data1)=6
#ncol(data1)=5



Row,Column1,XC60_Gent,XC60_China,Kuga,Mondeo
,String15,Float64,Float64,Float64,Float64
1,machine 1,0.0204601,0.0293755,0.0236603,0.0276006
2,mahicne 2,0.0260603,0.0251784,0.027873,0.0
3,machine 3,0.0230815,0.0267955,0.0224861,0.0285837
4,machine 4,0.0210469,0.0,0.0281093,0.024199
5,machine 5,0.0284882,0.0310221,0.024909,0.0279759
6,machine 6,0.024501,0.0278305,0.0256556,0.0316377


In [15]:
Random.seed!(1234) # set seed

num_products_p = ncol(data) - 1  #to not include the time columnindex
time_horizon_T = nrow(data)
product_names = names(data)[2:end]
num_machines_M=nrow(data1)
println(product_names)

#Initialise demand
demand_D = [] #Initialise 
for col in 2:ncol(data) #Extract each column as a seperate array and store it
    push!(demand_D, data[:,col])
end
println(demand_D)

#Initialise workdays 
workdays_n = [] #Initialise 
for row in 1:time_horizon_T
    push!(workdays_n, rand(20:23)) #push a random workday value between 20 to 23
end
println(workdays_n)

#Initialise productivity
productivity_K = [] 
for col in 2:ncol(data)
    push!(productivity_K, rand(10:12))
end    
println(productivity_K)

# Initialise Costs
cost_hiring_cH = 5882.60    # Hiring cost per worker
cost_firing_cF = 857.23    # Firing cost per worker
cost_inventory_cI = 9.05 # Inventory holding cost per unit
cost_labour_cR = 233.30  # Cost of Labour per production unit
cost_overtime_cO = 349.89 # Cost of Overtime per overtime unit
cost_backlogging_cB = 135.31  #Cost of Backlogging per overtime unit 

#Initialise time taken
timetaken_τ=[]
for col in 2:ncol(data1)
    push!(timetaken_τ, data1[:,col])
end
println(timetaken_τ)

#code ends here

["XC60_Gent", "XC60_China", "Kuga", "Mondeo"]
Any[[5668, 3916, 5312, 6720, 4092, 3108, 4656, 4772, 3408, 2936, 8284, 4516], [5952, 3744, 1552, 3032, 2372, 2292, 1568, 612, 3988, 2540, 6680, 3260], [2012, 2128, 2632, 1740, 1540, 2292, 2920, 3256, 2288, 2424, 2228, 2660], [924, 800, 964, 768, 648, 848, 828, 764, 720, 448, 700, 568]]
Any[21, 22, 20, 23, 21, 21, 23, 23, 21, 22, 22, 22]
Any[10, 10, 11, 12]
Any[[0.020460072, 0.026060266, 0.023081476, 0.021046905, 0.028488239, 0.024500974], [0.029375538, 0.02517837, 0.026795545, 0.0, 0.031022069, 0.02783053], [0.023660263, 0.027872984, 0.022486077, 0.028109273, 0.024908955, 0.025655647], [0.027600623, 0.0, 0.028583745, 0.024199002, 0.027975851, 0.031637681]]


In [61]:
# Initialize the model
model = Model(Gurobi.Optimizer)

# Variables
@variable(model, workerlevel_W[1:num_products_p,1:time_horizon_T] >= 0)       # Workers
@variable(model, hired_H[1:num_products_p,1:time_horizon_T] >= 0)       # Hired workers
@variable(model, fired_F[1:num_products_p,1:time_horizon_T] >= 0)       # Fired workers
@variable(model, inventory_I[1:num_products_p,1:time_horizon_T] >= 0)       # Inventory as integer variables
@variable(model, production_P[1:num_products_p,1:time_horizon_T] >= 0)       # Production
@variable(model, overtime_O[1:num_products_p,1:time_horizon_T] >= 0)       # Overtime
@variable(model, backlogging_B[1:num_products_p,1:time_horizon_T] >= 0)        # Production

# Objective function: Minimize total cost
@objective(model, Min, sum(cost_hiring_cH*hired_H + cost_firing_cF*fired_F 
+ cost_inventory_cI*inventory_I + cost_labour_cR*production_P 
+ cost_overtime_cO*overtime_O + cost_backlogging_cB*backlogging_B))

# Constraints

#only for period 1, W annd I-B eqns
for p in 1:num_products_p
    @constraint(model, workerlevel_W[p,1] == hired_H[p,1] - fired_F[p,1])
    @constraint(model, inventory_I[p,1] - backlogging_B[p,1] == production_P[p,1] - demand_D[p][1])
end    

#P equations
for p in 1:num_products_p
    for t in 2:time_horizon_T
        @constraint(model, workerlevel_W[p,t] 
        == workerlevel_W[p,t-1] + hired_H[p,t] - fired_F[p,t])
        @constraint(model, inventory_I[p,t] - backlogging_B[p,t] 
        == inventory_I[p,t-1] - backlogging_B[p,t-1] 
        + production_P[p,t] - demand_D[p][t])
    end
end

#for rest of period, W and I-B equations
for p in 1:num_products_p
    for t in 1:time_horizon_T
        @constraint(model, production_P[p,t] 
        == productivity_K[p] * workdays_n[t] * workerlevel_W[p,t])
    
        total_time_taken = 0.0
        for m in 1:num_machines_M
            total_time_taken += timetaken_τ[p][m] * production_P[p, t]
        end
        @constraint(model, total_time_taken <= workerlevel_W[p, t] * workdays_n[t] * 8)
        #hired_H
    end    
end


# Solve the model
optimize!(model)

#retrieve Values
println("Objective value: ", objective_value(model))
println("Workers: ", round.(Int,value.(workerlevel_W)))
println("Hired: ", round.(Int,value.(hired_H)))
println("Fired: ", round.(Int,value.(fired_F)))
println("Inventory: ", round.(Int,value.(inventory_I)))
println("Production: ", round.(Int,value.(production_P)))
println("Overtime: ", round.(Int,value.(overtime_O)))
println("Backlogging: ", round.(Int,value.(backlogging_B)))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-05
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 22.6.0 22G91)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 192 rows, 336 columns and 612 nonzeros
Model fingerprint: 0xd301ee74
Coefficient statistics:
  Matrix range     [1e-01, 3e+02]
  Objective range  [9e+00, 6e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 8e+03]
Presolve removed 100 rows and 144 columns
Presolve time: 0.00s
Presolved: 92 rows, 192 columns, 608 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.6166818e+35   8.843725e+31   1.616682e+05      0s
     129    3.0473986e+07   0.000000e+00   0.000000e+00      0s

Solved in 129 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.047398650e+07

User-callback calls 209, time in user-callback 0.00 sec
Objective value: 3.04

Base.Meta.ParseError: ParseError:
# Error @ /Users/axgelx/MSO ESA prj/Backend.ipynb:64:1
println("Backlogging: ", round.(Int,value.(backlogging_B)))
end
└─┘ ── invalid identifier